# Improved Fully Connected Layers
---

In [ ]:
import numpy as np
from collections import OrderedDict
from keras.datasets import mnist
import matplotlib.pyplot as plt

In [ ]:
# optimizers
class SGD:
    """
    stochastic gradient descent
    lr: learning rate hyperparameter
    """
    
    def __init__(self, lr = 0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

class Momentum:
    """
    Momentum SGD
    lr: learning rate hyperparameter (eta in the above)
    momentum: momentum hyperparameter (alpha in the above)
    """
    
    def __init__(self, lr = 0.01, momentum = 0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                # initial momentum -> 0
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

class Nesterov:
    """
    Nesterov's Accelerated Gradient
    More aggressive than Momentum optimizer
    However, the backpropagation process needs to be modified
    to implement NAG. See the formula, f, above.
    This is the reason why it can hardly be applied to deep learning.
    Instead, we can use Bengio's approximation like below.
    lr: learning rate hyperparameter (eta in the above)
    momentum: momentum hyperparameter (alpha in the above)
    """
    
    def __init__(self, lr = 0.01, momentum = 0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            params[key] += self.momentum * self.momentum * self.v[key]
            params[key] -= (1 + self.momentum) * self.lr * grads[key]
            self.v[key] *= self.momentum
            self.v[key] -= self.lr * grads[key]

class AdaGrad:
    """
    AdaGrad Optimizer
    lr: a learning rate component (eta in the above)
    self.h: inversely related to lr
    """
    def __init__(self, lr = 0.01):
        self.lr = lr # eta in the above
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.h[key] += grads[key] * grads[key] # Hadamard product
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key] + 1e-7))
            
class RMSProp:
    """
    RMSProp Optimizer
    lr: learning rate hyperparameter (eta in the above)
    gamma: decaying rate hyperparameter (gamma in the above)
    """
    def __init__(self, lr = 0.01, gamma = 0.99):
        self.lr = lr
        self.gamma = gamma
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.h[key] = self.gamma * self.h[key] \
            + (1- self.gamma) * grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key] + 1e-7))
            
class Adam:
    """
    Adam Optimizer
    lr: a learning rate component (eta in the above)
    beta1: a hyperparameter for the Momentum Part
    beta2: a hyperparameter for the RMSProp Part (varying learning rate)
    """
    
    def __init__(self, lr = 0.001, beta1 = 0.9, beta2 = 0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
                
        self.iter += 1
        # modified learning rate (to replace m and v with m_hat and v_hat)
        lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / \
            (1.0 - self.beta1**self.iter)
        for key in params.keys():
            # Update the Momentum hyperparameter.
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            # Update the RMSProp hyperparameter.
            # Hadamard product can be easily implemented via NumPy operations.
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)

In [ ]:
# activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(x))

def relu(x):
    return np.maximum(0, x)

def tanh(x):
    return np.tanh(x)

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis = 0)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0)
        
        return y.T 

    x = x - np.max(x) # preventing overflow
    
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # Convert into groundtruth labels if training data are one-hot encoded.
    if t.size == y.size:
        t = t.argmax(axis = 1)
             
    batch_size = y.shape[0]
    
    return -np.sum(np.log(
        y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [ ]:
# activation classes
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

class ReLU:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [ ]:
# necessary functions and classes
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # gradients
        self.dW = None
        self.db = None

    def forward(self, x):
        # for tensor operations
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        # for tensor operations
        dx = dx.reshape(*self.original_x_shape)
        
        return dx

class SoftMaxWithLoss:
    def __init__(self):
        self.loss = None # loss function
        self.y = None    # SoftMax output
        self.t = None    # groundtruth (one-hot encoded)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # if one-hot encoded groundtruth
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags = ['multi_index'], op_flags = ['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x + h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x - h)
        grad[idx] = (fxh1 - fxh2) / (2 * h)
        
        x[idx] = tmp_val # Restore values.
        it.iternext()   
        
    return grad

def shuffle_dataset(x, t):
    permutation = np.random.permutation(x.shape[0])
    x = x[permutation, :] if x.ndim == 2 else x[permutation, :, :, :]
    t = t[permutation]

    return x, t

In [ ]:
class BatchNormalization:
    """
    http://arxiv.org/abs/1502.03167
    """
    def __init__(self, gamma, beta, momentum = 0.9,
                 running_mean = None, running_var = None):
        self.gamma = gamma
        self.beta = beta
        self.momentum = momentum
        self.input_shape = None # CNN: 4-dimensional, FCL: 2-dimensional
        
        # for back prop
        self.batch_size = None
        self.xc = None
        self.std = None
        self.dgamma = None
        self.dbeta = None
        
        # for test (validation)
        self.running_mean = running_mean
        self.running_var = running_var
        
    def forward(self, x, train_flg = True):
        self.input_shape = x.shape
        if x.ndim != 2: # in case of CNN
            N, C, H, W = x.shape
            x = x.reshape(N, -1)
            
        out = self.__forward(x, train_flg)
        
        return out.reshape(*self.input_shape) # Restore the original shape.
    
    def __forward(self, x, train_flg):
        if self.running_mean is None:
            N, D = x.shape
            self.running_mean = np.zeros(D)
            self.running_var = np.zeros(D)
            
        if train_flg: # training
            mu = x.mean(axis = 0)
            xc = x - mu
            var = np.mean(xc**2, axis = 0)
            std = np.sqrt(var + 10e-7)
            xn = xc / std
            
            self.batch_size = x.shape[0]
            self.xc = xc
            self.xn = xn
            self.std = std
            self.running_mean = self.momentum * self.running_mean +\
            (1 - self.momentum) * mu
            self.running_var = self.momentum * self.running_var +\
            (1 - self.momentum) * var
        else: # validation
            xc = x - self.running_mean
            xn = xc / ((np.sqrt(self.running_var + 10e-7)))
            
        out = self.gamma * xn + self.beta
        
        return out
    
    def backward(self, dout):
        if dout.ndim != 2:
            N, C, H, W = dout.shape
            dout = dout.reshape(N, -1)
            
        dx = self.__backward(dout)
        dx = dx.reshape(*self.input_shape)
        
        return dx
    
    def __backward(self, dout): # If repeat node, sum!!!
        dbeta = dout.sum(axis = 0)
        dgamma = np.sum(self.xn * dout, axis = 0)
        dxn = self.gamma * dout
        dxc = dxn / self.std
        dstd = -np.sum((dxn * self.xc) / (self.std * self.std), axis = 0)
        dvar = 0.5 * dstd / self.std
        dvar /= self.batch_size # mean node
        dxc += dvar * (2.0 * self.xc)
        dmu = -np.sum(dxc, axis = 0)
        dx = dxc + (dmu / self.batch_size) # the 2nd factor: mean node
        
        self.dgamma = dgamma
        self.dbeta = dbeta
        
        return dx

In [ ]:
class Dropout:
    """
    http://arxiv.org/abs/1207.0580
    """
    def __init__(self, dropout_ratio = 0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x, train_flg=True):
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)

    def backward(self, dout):
        return dout * self.mask

the number of Affine = the number of hidden + 1 + 1
<br>$\rightarrow$ 1: Input Layer, Output Layer
<br>
the number of Batch Normalization = the number of hidden + 1
<br>$\rightarrow$ 1: Input Layer
<br>
the number of Dropout = the number of hidden + 1
<br>$\rightarrow$ 1: Input Layer

In [ ]:
class MultiLayerNetExtend:
    """ Fully Connected Layer
    Parameters
    ----------
    input_size : 784 if MNIST
    hidden_size_list : the number of neurons in a hidden layer（e.g. [100, 100, 100]）
    output_size : 10 if MNIST
    activation : 'ReLU' or 'Sigmoid'
    weight_init_std : e.g. 0.01
        'ReLU' or 'He' -> Set He Initialization.
        'Sigmoid' or 'Xavier' -> Set Xavier Initialization.
    weight_decay_lambda : regularization parameter
    use_dropout : True or False
    dropout_ratio : Dropout Ratio
    use_batchNorm : True or False
    """
    def __init__(self, input_size, hidden_size_list, output_size,
                 activation = 'ReLU', weight_init_std = 'ReLU',
                 weight_decay_lambda = 0, use_dropout = False,
                 dropout_ratio = 0.5, use_batchnorm = False):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size_list = hidden_size_list
        self.hidden_layer_num = len(hidden_size_list)
        self.use_dropout = use_dropout
        self.weight_decay_lambda = weight_decay_lambda
        self.use_batchnorm = use_batchnorm
        self.params = {}

        # weight initialization
        self.__init_weight(weight_init_std)

        # the order of layers: Affine + Batch Norm + Activation + Dropout
        # creating hidden layers (Affine + activation per hidden layer)
        activation_layer = {'Sigmoid': Sigmoid, 'ReLU': ReLU}
        self.layers = OrderedDict()
        
        # input layer and hidden layers
        for idx in range(1, self.hidden_layer_num + 1):
            self.layers['Affine' + str(idx)] = Affine(
                self.params['W' + str(idx)],
                self.params['b' + str(idx)])
            
            if self.use_batchnorm:
                self.params['gamma' + str(idx)] = np.ones(
                    hidden_size_list[idx -1])
                self.params['beta' + str(idx)] = np.zeros(
                    hidden_size_list[idx - 1])
                self.layers['BatchNorm' + str(idx)] = BatchNormalization(
                    self.params['gamma' + str(idx)],
                    self.params['beta' + str(idx)])
            
            self.layers['Activation_function' + str(idx)] = activation_layer[activation]()
            
            if self.use_dropout:
                self.layers['Dropout' + str(idx)] = Dropout(dropout_ratio)

        # the output layer's Affine
        idx = self.hidden_layer_num + 1
        self.layers['Affine' + str(idx)] = Affine(
            self.params['W' + str(idx)],
            self.params['b' + str(idx)])

        # cross entropy (not SoftMax function)
        self.last_layer = SoftMaxWithLoss()

    def __init_weight(self, weight_init_std):
        """
        Weight Initialization Parameters
        ----------
        weight_init_std : e.g. 0.01
            'ReLU' or 'He' -> Set He Initialization.
            'Sigmoid' or 'Xavier' -> Set Xavier Initialization.
        """
        all_size_list = [self.input_size] + self.hidden_size_list + [self.output_size]
        # range: We do not need to consider the output layer's
        # weight initialization.
        for idx in range(1, len(all_size_list)):
            scale = weight_init_std
            if str(weight_init_std).lower() in ('relu', 'he'):
                # recommended if ReLU
                scale = np.sqrt(2.0 / all_size_list[idx - 1])
            elif str(weight_init_std).lower() in ('sigmoid', 'xavier'):
                # recommended if Sigmoid
                scale = np.sqrt(1.0 / all_size_list[idx - 1])
            self.params['W' + str(idx)] = scale * np.random.randn(
                all_size_list[idx - 1], all_size_list[idx])
            self.params['b' + str(idx)] = np.zeros(all_size_list[idx])

    def predict(self, x, train_flg = False):
        for key, layer in self.layers.items():
            if "Dropout" in key or "BatchNorm" in key:
                x = layer.forward(x, train_flg)
            else:
                # You need to additionally apply a SoftMax function
                # to get predicted values.
                x = layer.forward(x)

        return x

    def loss(self, x, t, train_flg = False):
        y = self.predict(x, train_flg)

        weight_decay = 0
        # self.hidden_layer_num + 2: hidden layers
        # + (input layer + output layer)
        for idx in range(1, self.hidden_layer_num + 2):
            W = self.params['W' + str(idx)]
            weight_decay += 0.5 * self.weight_decay_lambda *\
            np.sum(W ** 2)

        return self.last_layer.forward(y, t) + weight_decay

    def accuracy(self, x, t):
        y = self.predict(x, train_flg = False)
        y = np.argmax(y, axis = 1)
        # unless one-hot encoded groundtruth
        # ex) one-hot encoded: t = [[0, 0, 1, 0, ..., 0],
        #                           [1, 0, 0, 0, ..., 0],
        #                           [0, 1, 0, 0, ..., 0]] -> t.ndim = 2
        # ex) not one-hot encoded: t = [2, 0, 1] -> t.ndim = 1
        if t.ndim != 1: t = np.argmax(t, axis = 1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t, train_flg = True)

        grads = {}
        for idx in range(1, self.hidden_layer_num + 2):
            grads['W' + str(idx)] = numerical_gradient(
                loss_W, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(
                loss_W, self.params['b' + str(idx)])
            
            if self.use_batchnorm and idx != self.hidden_layer_num + 1:
                grads['gamma' + str(idx)] = numerical_gradient(
                    loss_W, self.params['gamma' + str(idx)])
                grads['beta' + str(idx)] = numerical_gradient(
                    loss_W, self.params['beta' + str(idx)])

        return grads

    # back propagation
    def gradient(self, x, t):
        # forward
        self.loss(x, t, train_flg = True)
        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # Store results.
        grads = {}
        for idx in range(1, self.hidden_layer_num + 2):
            # L2-Regularization assumed (differentiated by weights)
            grads['W' + str(idx)] = self.layers['Affine' + str(idx)].dW +\
            self.weight_decay_lambda * self.layers['Affine' + str(idx)].W
            grads['b' + str(idx)] = self.layers['Affine' + str(idx)].db
            
            if self.use_batchnorm and idx != self.hidden_layer_num + 1:
                grads['gamma' + str(idx)] = self.layers['BatchNorm' + str(idx)].dgamma
                grads['beta' + str(idx)] = self.layers['BatchNorm' + str(idx)].dbeta

        return grads

In [ ]:
# input data
(x_train, t_train), (x_test, t_test) = mnist.load_data()

x_train = (x_train - 0.0) / (255.0 - 0.0)
x_train = x_train.reshape(-1, 784)
x_test = (x_test - 0.0) / (255.0 - 0.0)
x_test = x_test.reshape(-1, 784)

For the further learning, study Mixture Density Networks, and Bayesian Neural Networks in C. Bishop (2006).